In [1]:
import pip
import numpy as np

In [2]:
pip.main(['install', 'mip'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Defaulting to user installation because normal site-packages is not writeable


0

In [3]:
from mip import Model, xsum, maximize, BINARY

p = [10, 13, 18, 31, 7, 15]
w = [11, 15, 20, 35, 10, 33]
c, I = 47, range(len(w))

m = Model("knapsack")

x = [m.add_var(var_type=BINARY) for i in I]

m.objective = maximize(xsum(p[i] * x[i] for i in I))

m += xsum(w[i] * x[i] for i in I) <= c

m.optimize()

selected = [i for i in I if x[i].x >= 0.99]
print("selected items: {}".format(selected))

Using Python-MIP package version 1.13.0
selected items: [0, 3]


In [26]:
from mip import Model, xsum, maximize, BINARY

nt = 18
k  = 2

m = Model("Sports League")

xw = []
xd = []
xl = []

for i in range(nt):
    xw.append([])
    xd.append([])
    xl.append([])
    for j in range(nt):
        if i!=j:
            xw[i].append(m.add_var(var_type=BINARY))
            xd[i].append(m.add_var(var_type=BINARY))
            xl[i].append(m.add_var(var_type=BINARY))
            m += (xw[i][j] + xd[i][j] + xl[i][j]) == 1
        else:
            xw[i].append([])
            xd[i].append([])
            xl[i].append([])

for t in range(nt-1):
    m +=    xsum( 3*xl[i][t]   + xd[i][t]   + 
                  3*xw[t][i]   + xd[t][i]  for i in range(nt) if i!=t) <=\
            xsum( 3*xl[i][t+1] + xd[i][t+1] + 
                  3*xw[t+1][i] + xd[t+1][i]  for i in range(nt) if i!=(t+1))

m.objective = maximize(xsum( 3*xl[i][k] + xd[i][k] + 
                             3*xw[k][i] + xd[k][i]  for i in range(nt) if i!=k))
m.optimize() 

pk = np.sum([3*int(xl[i][k].x) + int(xd[i][k].x) + 
           3*int(xw[k][i].x) + int(xd[k][i].x) for i in range(nt) if i!=k])

print("Team k={} has {} points and will still relegate.".format(k,pk)+\
      "Therefore a team is safe if it has at least {} points".format(pk+1))


for t in range(nt):
    team = [i for i in range(nt) if i!=t]
    resow = [int(xl[i][t].x) for i in range(nt) if i!=t]
    resod = [int(xd[i][t].x) for i in range(nt) if i!=t]
    resol = [int(xw[i][t].x) for i in range(nt) if i!=t]
    
    reshw = [int(xw[t][i].x) for i in range(nt) if i!=t]
    reshd = [int(xd[t][i].x) for i in range(nt) if i!=t]
    reshl = [int(xl[t][i].x) for i in range(nt) if i!=t]
    
    resh = np.array([team, reshw, reshd, reshl])
    reso = np.array([team, resow, resod, resol])
    
    pt =np.sum(3*resh[1,:])+np.sum(resh[2,:]) + np.sum(3*reso[1,:])+np.sum(reso[2,:]) 
    
    resh[1] +=2*resh[2]+3*resh[3] 
    reso[1] +=2*reso[2]+3*reso[3] 
    print(t)
    print(resh[1])
    print(reso[1])
    print(pt)

Team k=2 has 57 points and will still relegate.Therefore a team is safe if it has at least 58 points
0
[2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
[2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
2
1
[2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
[2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
2
2
[1 1 1 3 1 3 1 1 1 1 1 3 3 3 1 1 3]
[1 1 1 1 3 1 1 1 1 3 3 3 3 3 3 3 3]
57
3
[1 1 3 1 3 1 1 1 1 3 3 1 1 1 3 3 1]
[1 1 3 1 1 3 1 1 1 1 3 3 3 3 3 3 3]
57
4
[1 1 3 3 1 3 3 1 3 1 1 1 1 3 3 3 3]
[1 1 1 3 1 1 1 1 1 1 1 1 3 3 3 3 3]
57
5
[1 1 1 3 3 1 3 3 1 1 3 3 3 1 1 3 3]
[1 1 3 1 3 1 1 1 1 1 1 1 1 3 3 3 3]
57
6
[1 1 3 1 3 3 3 3 3 1 1 1 1 1 3 3 3]
[1 1 1 3 1 3 1 1 1 1 1 3 1 1 3 3 3]
57
7
[1 1 3 3 3 3 3 1 3 3 1 3 3 3 3 1 1]
[1 1 3 3 1 1 1 1 1 1 1 1 1 1 1 3 3]
57
8
[1 1 3 3 3 3 3 3 1 1 3 1 1 1 1 3 3]
[1 1 3 3 3 1 1 3 1 1 1 1 1 3 1 1 3]
57
9
[1 1 3 3 3 3 3 3 3 1 1 3 1 3 3 1 1]
[1 1 3 3 1 3 1 1 3 1 1 1 1 1 3 1 1]
57
10
[1 1 1 3 3 3 3 3 3 3 1 1 3 1 1 3 1]
[1 1 3 1 3 3 3 1 3 3 1 1 1 1 1 1 1]
57
11
[1 1 1 1 3 3 3 3 3 3 3 1 3 1 1 1 3]
[1 1 3 1 3 1 3